# Additional Data Sources

## Historical Weather Data from NOAA

In [53]:
weather = pd.read_csv(f"{data_path}/raw/weather/weather.csv")
weather.columns = [x.lower() for x in list(weather.columns)]
weather['date'] = pd.to_datetime(weather['date'])
weather = weather.set_index('date')
station = "USC00286964"
weather = weather[weather.station == station]
weather.head()

,station,name,latitude,longitude,elevation,awnd,dapr,mdpr,pgtm,prcp,snow,snwd,tavg,tmax,tmin,tobs,wdf2,wdf5,wesd,wesf,wsf2,wsf5,wt01,wt02,wt03,wt04,wt05,wt06,wt08,wt11
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,USC00286964,PHILADELPHIA MT. HOLLY WEATHER FORECAST OFFICE...,40.01369,-74.81753,19.2,NaN,NaN,NaN,NaN,0.00,0.0,0.0,NaN,46.0,37.0,37.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,USC00286964,PHILADELPHIA MT. HOLLY WEATHER FORECAST OFFICE...,40.01369,-74.81753,19.2,NaN,NaN,NaN,NaN,0.00,0.0,0.0,NaN,41.0,21.0,22.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,USC00286964,PHILADELPHIA MT. HOLLY WEATHER FORECAST OFFICE...,40.01369,-74.81753,19.2,NaN,NaN,NaN,NaN,0.12,0.0,0.0,NaN,50.0,21.0,43.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-04,USC00286964,PHILADELPHIA MT. HOLLY WEATHER FORECAST OFFICE...,40.01369,-74.81753,19.2,NaN,NaN,NaN,NaN,0.26,0.0,0.0,NaN,48.0,43.0,48.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-05,USC00286964,PHILADELPHIA MT. HOLLY WEATHER FORECAST OFFICE...,40.01369,-74.81753,19.2,NaN,NaN,NaN,NaN,0.07,0.0,0.0,NaN,53.0,38.0,38.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Google's COVID-19 Community Mobility Reports

In [202]:
city_mobility = pd.read_csv(f"{data_path}/raw/mobility/Google Mobility - City - Daily.csv").merge(pd.read_csv(f"{data_path}/raw/geo/GeoIDs - City.csv"), on='cityid')
county_mobility = pd.read_csv(f"{data_path}/raw/mobility/Google Mobility - County - Daily.csv").merge(pd.read_csv(f"{data_path}/raw/geo/GeoIDs - County.csv"), on='countyfips')
state_mobility = pd.read_csv(f"{data_path}/raw/mobility/Google Mobility - State - Daily.csv").merge(pd.read_csv(f"{data_path}/raw/geo/GeoIDs - State.csv"), on='statefips')
national_mobility = pd.read_csv(f"{data_path}/raw/mobility/Google Mobility - National - Daily.csv")
city_mobility.head()

,year,month,day,cityid,gps_retail_and_recreation,gps_grocery_and_pharmacy,gps_parks,gps_transit_stations,gps_workplaces,gps_residential,gps_away_from_home,cityname,stateabbrev,statename,statefips,city_pop2019
0,2020,2,24,1,0.00571,-0.00286,.0714,.00286,0.0214,-0.00143,0.000625,Los Angeles,CA,California,6,10039107
1,2020,2,25,1,0.01000,-0.00143,.0743,.00429,0.0229,-0.00286,0.002020,Los Angeles,CA,California,6,10039107
2,2020,2,26,1,0.01570,0.00286,.0714,.00571,0.0243,-0.00429,0.003620,Los Angeles,CA,California,6,10039107
3,2020,2,27,1,0.01860,0.00571,.0757,.00714,0.0243,-0.00429,0.003620,Los Angeles,CA,California,6,10039107
4,2020,2,28,1,0.02290,0.01140,.08,.01,0.0257,-0.00571,0.005000,Los Angeles,CA,California,6,10039107


In [204]:
combine_date_parts = lambda x: "-".join([str(x.year), str(x.month).zfill(2), str(x.day).zfill(2)])
city_mobility['Date'] = city_mobility.apply(combine_date_parts, axis=1)

In [224]:
city_mobility['Date'] = pd.to_datetime(city_mobility['Date'])
df = city_mobility[city_mobility.cityname == 'Philadelphia'].merge(sg.calc_true_visits_by_day(get_sg_data("wawa", "oct").iloc[0]).reset_index(), how='right', left_on='Date', right_on='index')
pop = 1584000 * 0.25
df['mobility'] =  pop + (pop * df['gps_away_from_home'])
df[['Date', 'true_visits', 'mobility']].set_index('Date').corr()

,true_visits,mobility
true_visits,1.000000,0.479887
mobility,0.479887,1.000000


In [237]:
city_mobility['Date'] = pd.to_datetime(city_mobility['Date'])
df = city_mobility[city_mobility.cityname == 'Philadelphia'].merge(sg.calc_true_visits_by_day(get_sg_data("wawa", "oct").iloc[0]).reset_index(), how='right', left_on='Date', right_on='index')
df[['true_visits', 'gps_retail_and_recreation', 'gps_grocery_and_pharmacy', 'gps_parks', 'gps_transit_stations', 'gps_workplaces', 'gps_residential', 'gps_away_from_home']].corr()

,true_visits,gps_retail_and_recreation,gps_grocery_and_pharmacy,gps_workplaces,gps_residential,gps_away_from_home
true_visits,1.000000,0.587968,0.608726,0.412742,-0.486473,0.479887
gps_retail_and_recreation,0.587968,1.000000,0.972060,0.204902,-0.958927,0.967373
gps_grocery_and_pharmacy,0.608726,0.972060,1.000000,0.313941,-0.932271,0.932119
gps_workplaces,0.412742,0.204902,0.313941,1.000000,-0.149597,0.170817
gps_residential,-0.486473,-0.958927,-0.932271,-0.149597,1.000000,-0.988547
gps_away_from_home,0.479887,0.967373,0.932119,0.170817,-0.988547,1.000000
